In [1]:
%load_ext Cython
import numpy as np


# cdef & cpdef

In [2]:
def function(x, y):
    return x**y

In [3]:
%timeit function(5, 6)

The slowest run took 9.58 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 311 ns per loop


In [4]:
%%cython

cdef cdef_funtion(x, y):
    return x**y



In [5]:
cdef_funtion(5, 6)

NameError: ignored

Cdef functions are not callable from non cython cells.

In [5]:
%%cython

cpdef cpdef_function(x,y):
    return x**y

In [6]:
%timeit cpdef_function(5,6)

The slowest run took 13.09 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 291 ns per loop


Not really faster. So introduce datatypes!

In [7]:
%%cython

cpdef int cpdef_dtypes_function(int x, int y):
    return x**y

In [8]:
%timeit cpdef_dtypes_function(5,6)

The slowest run took 22.60 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 110 ns per loop


# Cython and Numpy Arrays

In [9]:
%%cython
import numpy as np

arr = np.arange(9, dtype=np.dtype("i")).reshape((3, 3))
print(arr)

cdef int [:, :] arr_view = arr
print(np.asarray(arr_view))

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [10]:
%%cython 
import numpy as np

cdef int [:, :] arr_view = np.arange(9, dtype=np.dtype("i")).reshape((3, 3))
print(np.asarray(arr_view))
print(np.asarray(arr_view[1,1]))
arr_view[1,1] = 10
print(np.asarray(arr_view[1,1]))
arr_view[:,:] = 5
print(np.asarray(arr_view))


[[0 1 2]
 [3 4 5]
 [6 7 8]]
4
10
[[5 5 5]
 [5 5 5]
 [5 5 5]]


## Looping over arrays

In [11]:
x = np.arange(100, dtype=np.dtype("i")).reshape((10, 10))
y = np.full((10,10), 2, dtype=np.dtype("i"))

In [12]:
def array_op(x,y):
    result = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result
    

In [13]:
%timeit result = array_op(x,y)

1000 loops, best of 5: 292 µs per loop


In [14]:
%%cython 
import numpy as np

cpdef cdef_array_op(x, y):
    result = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result

In [15]:
%timeit cdef_array_op(x,y)

1000 loops, best of 5: 279 µs per loop


In [16]:
%%cython 
import numpy as np

cpdef int[:,:] typed_array_op(int[:,:] x, int[:,:] y):
    cdef int[:,:] result = np.zeros((x.shape[0], x.shape[1]), dtype = np.dtype("i"))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result

In [17]:
%timeit typed_array_op(x,y)

The slowest run took 10.11 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 5.93 µs per loop


In [18]:
%%cython 
import numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef int[:,:] typed_unsafe_array_op(int[:,:] x, int[:,:] y):
    cdef int[:,:] result = np.zeros((x.shape[0], x.shape[1]), dtype = np.dtype("i"))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result


In [19]:
%timeit typed_unsafe_array_op(x,y)

The slowest run took 9.81 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 5.71 µs per loop


# Cython and Pandas

We downloaden eerst wat data...

In [20]:
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt    
!curl -L -b cookies.txt -o 'weatherAUS.zip' 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd&confirm='$(<confirm.txt)
!unzip weatherAUS.zip
!rm -f confirm.txt cookies.txt weatherAUS.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    200      0 --:--:--  0:00:01 --:--:--   200
100 3781k    0 3781k    0     0  1560k      0 --:--:--  0:00:02 --:--:-- 1560k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    168      0 --:--:--  0:00:02 --:--:--   167
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
100 3781k    0 3781k    0     0  1091k      0 --:--:--  0:00:03 --:--:--  300M
Archive:  weatherAUS.zip
  inflating: weatherAUS.csv          


In [21]:
import pandas as pd

In [22]:
df = pd.read_csv('weatherAUS.csv')

In [23]:
df.head()
# TODO: NaN's aanvullen

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [24]:
def average_value(values):
  return np.mean(values)


In [25]:
%timeit df.head(10).apply(lambda x: average_value(x[11:13]), axis=1)


100 loops, best of 5: 2.54 ms per loop


In [26]:
%%cython
import numpy as np

cpdef c_average_value(values):
  return np.mean(values)


In [27]:
%timeit df.head(10).apply(lambda x: c_average_value(x[11:13]), axis=1)

100 loops, best of 5: 2.71 ms per loop


In [28]:
%%cython
import numpy as np
cimport numpy as np

cpdef c_average_value_typed(int[:] values):
  return np.mean(values)

In [29]:
%timeit df.head(10).apply(lambda x: c_average_value_typed(x[11:13].to_numpy(dtype=np.dtype('i'))), axis=1)

1000 loops, best of 5: 1.5 ms per loop


In [45]:
%%cython
cimport cython
cimport numpy as np
import numpy as np

cpdef c_average_value_typed(int[:] values):
  return np.mean(values)

cpdef c_apply_average_value(int[:,:] df_cols):
  n = df_cols.shape[0]
  cdef double[:] res = np.empty(n)
  for i in range(n):
    res[i] = c_average_value_typed(df_cols[i])

  return res

In [46]:
%timeit c_apply_average_value(df.head(10)[df.columns[11:13]].to_numpy(dtype=np.dtype('i'))) 

The slowest run took 7.44 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 808 µs per loop
